In [1]:
% matplotlib inline

import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import csv

In [15]:
def scale_rotate_translate_img(img, angle=0, scale=1.0, tx=0, ty=0):
    """
    escalar, rotar y trasladar usando la matriz de transformación
    """
    (h, w) = img.shape[:2]
    (cX, cY) = (w // 2, h // 2)
    # cálculo de la matriz de transformación
    M = cv2.getRotationMatrix2D((cX, cY), angle, scale)
    T = [[0, 0, tx],[0, 0, ty]]
    return cv2.warpAffine(img, M + T, (w, h))

def combine(palm_path, mask_path, bk_path, angle_deg=0, scale=1.0, tx=0, ty=0, bk_focus_blur_kernel=(0, 0)):
    # leer palma y máscara
    palm = cv2.imread(palm_path)
    palm = cv2.cvtColor(palm, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(mask_path)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    # erosionamos un poco los bordes blancos
    mask = cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)))

    # leer nuevo fondo
    bk = cv2.imread(bk_path)
    bk = cv2.cvtColor(bk, cv2.COLOR_BGR2RGB)
    bk = cv2.resize(bk, (palm.shape[1],palm.shape[0]))
    
    # aplicar transformación para palma y máscara
    palm = scale_rotate_translate_img(palm, angle_deg, scale, tx, ty)
    mask = scale_rotate_translate_img(mask, angle_deg, scale, tx, ty)
    
    # suavizar máscara para que el cambio entre palma y nuevo fondo sea suave
    border_size_unit = 5
    mask = mask.astype(float)
    mask = cv2.GaussianBlur(mask, (np.uint8(border_size_unit*scale/2)*2+1, np.uint8(border_size_unit*scale/2)*2+1), 0)
    mask3c = np.zeros(palm.shape, float)
    mask3c[:,:,0] = mask
    mask3c[:,:,1] = mask
    mask3c[:,:,2] = mask    
    
    # combinación con suavización (alpha blending)
    palm = palm.astype(float)
    bk = bk.astype(float)
    mask3c = mask3c.astype(float)/255.0
    palm = cv2.multiply(mask3c, palm)  # F' = alpha * F
    bk = cv2.multiply(1.0 - mask3c, bk)  # B' = (1-alpha) * B
    combined = cv2.add(palm, bk)  # I = F' + B'
    return combined, mask

In [28]:
# dataset de nuevos fondos
bk_dir = "/Datasets/open-images/validation/"
bk_paths = [bk_dir + f for f in os.listdir(bk_dir)]

# dataset de palmas
palm_dir = "/Datasets/5k-palms/5k-palms/"
palm_paths = [palm_dir + f for f in os.listdir(palm_dir)]
palm_paths.sort()

# dataset de máscaras
mask_dir = "/Datasets/5k-palms/5k-palms-masks/"
mask_paths = [mask_dir + f for f in os.listdir(mask_dir)]
mask_paths.sort()

# nuevo dataset construido
dataset_dir = "/Datasets/5k-palms/40k/"
mask_dataset_dir = "/Datasets/5k-palms/40k-masks/"
with open(dataset_dir + "dataset.csv", "w") as csv_file:
    csv_writer = csv.writer(csv_file, delimiter=',')
    
    # pi = np.uint8(np.random.uniform(0, len(palm_paths)))
    for p in range(0, len(palm_paths)):
    # for p in range(pi, pi+2):
        palm_path = palm_paths[p]
        mask_path = mask_paths[p]

        palm = cv2.imread(palm_path)
        palm = cv2.cvtColor(palm, cv2.COLOR_BGR2RGB)
        
        num_samples_per_image = 8
        for i in range(0, num_samples_per_image):
            # leer fondo aleatorio
            bk_img_num = np.uint8(np.random.uniform(0, len(bk_paths)))
            bk_path = bk_paths[bk_img_num]
            print(bk_path)

            # generar parámetros aleatorios
            angle = np.random.uniform(-180, 180)
            scale = np.random.uniform(0.75, 1.75)
            tx = np.random.uniform(-palm.shape[1]//(4*scale), palm.shape[1]//(4*scale))  # sólo mover como máximo la "nava" parte
            ty = np.random.uniform(-palm.shape[0]//(4*scale), palm.shape[0]//(4*scale))  # sólo mover como máximo la "nava" parte
            focus_blur_size = np.uint8(np.random.uniform(0, 7))*2+1
            combined, combined_mask = combine(palm_path, mask_path, bk_path, angle, scale, tx, ty, (focus_blur_size,focus_blur_size))
            
            # escribir resultados
            combined = combined.astype(np.uint8)            
            combined_path = dataset_dir + os.path.basename(palm_path) +"."+str(i)+".png"
            cv2.imwrite(combined_path, cv2.cvtColor(combined, cv2.COLOR_RGB2BGR))
            combined_mask_path = mask_dataset_dir + os.path.basename(palm_path) +"."+str(i)+".png"
            cv2.imwrite(combined_mask_path, combined_mask)
            
            # escribir parámetros de entrada
            csv_writer.writerow([combined_path, combined_mask_path, palm_path, mask_path, bk_path, angle, scale, tx, ty, focus_blur_size])
            
    print(csv_file, " written")

/Datasets/open-images/validation/b8978c77a9367f6e.jpg
/Datasets/open-images/validation/66a8d1c74ad49ea1.jpg
/Datasets/open-images/validation/1e3791a88b656c6a.jpg
/Datasets/open-images/validation/153ccf228832ad30.jpg
/Datasets/open-images/validation/7025e644f4805d31.jpg
/Datasets/open-images/validation/ac18f2609189ed0c.jpg
/Datasets/open-images/validation/cba9b15d65e2bf25.jpg
/Datasets/open-images/validation/86f11d6200964c9f.jpg
/Datasets/open-images/validation/c917f04d2f7c8806.jpg
/Datasets/open-images/validation/f47e4757122337f8.jpg
/Datasets/open-images/validation/419d2e542228a165.jpg
/Datasets/open-images/validation/129d098f401181ed.jpg
/Datasets/open-images/validation/a403a0ffc5c655a3.jpg
/Datasets/open-images/validation/47eb4c87eb84f691.jpg
/Datasets/open-images/validation/42e96fdd7d038f79.jpg
/Datasets/open-images/validation/bcb2b1de10f8a2ae.jpg
/Datasets/open-images/validation/14deb9c4d2ab58c8.jpg
/Datasets/open-images/validation/341bafa58d2d369e.jpg
/Datasets/open-images/valida

/Datasets/open-images/validation/a510a4800789fba0.jpg
/Datasets/open-images/validation/66f018052f50e4f6.jpg
/Datasets/open-images/validation/ad9a3e709c2c7c5d.jpg
/Datasets/open-images/validation/d35531e146b18e65.jpg
/Datasets/open-images/validation/54e0022e04c69308.jpg
/Datasets/open-images/validation/c350de51c9fdbd43.jpg
/Datasets/open-images/validation/a4c7687ed90c5cf0.jpg
/Datasets/open-images/validation/a86c99772420c77e.jpg
/Datasets/open-images/validation/05ac515445e2b12f.jpg
/Datasets/open-images/validation/79b09f1821e48bd0.jpg
/Datasets/open-images/validation/bf8b72e9ae5bc47f.jpg
/Datasets/open-images/validation/2e9e55199995b910.jpg
/Datasets/open-images/validation/03bfd3f1488b47cb.jpg
/Datasets/open-images/validation/d1bfad28e7bbad62.jpg
/Datasets/open-images/validation/99f061be8c78fdd2.jpg
/Datasets/open-images/validation/87e55edd11b873e1.jpg
/Datasets/open-images/validation/402dd57d3615b9f7.jpg
/Datasets/open-images/validation/6a81196a04e03ea2.jpg
/Datasets/open-images/valida

/Datasets/open-images/validation/a8ef3ae2c618e2ae.jpg
/Datasets/open-images/validation/8455e1539fad43c0.jpg
/Datasets/open-images/validation/d8cf780534144096.jpg
/Datasets/open-images/validation/2cd624316fd6ff59.jpg
/Datasets/open-images/validation/c30ede4383ab1c8f.jpg
/Datasets/open-images/validation/3254cdeef31d6887.jpg
/Datasets/open-images/validation/cc6c037d61bd9e70.jpg
/Datasets/open-images/validation/487874143b9e2145.jpg
/Datasets/open-images/validation/617972ff9a2c82dd.jpg
/Datasets/open-images/validation/fb7e743fd383b034.jpg
/Datasets/open-images/validation/aa57164987eff0f6.jpg
/Datasets/open-images/validation/2cd624316fd6ff59.jpg
/Datasets/open-images/validation/66a8d1c74ad49ea1.jpg
/Datasets/open-images/validation/136f3389a1b90706.jpg
/Datasets/open-images/validation/c75dc9a0a71f5bf5.jpg
/Datasets/open-images/validation/79b09f1821e48bd0.jpg
/Datasets/open-images/validation/54e0022e04c69308.jpg
/Datasets/open-images/validation/b2fef7eca1352b4a.jpg
/Datasets/open-images/valida

/Datasets/open-images/validation/376c70019b86c05e.jpg
/Datasets/open-images/validation/dcbc80ec88ff4c3e.jpg
/Datasets/open-images/validation/dfd4b514fd717263.jpg
/Datasets/open-images/validation/79d8dab0d7006ae8.jpg
/Datasets/open-images/validation/57addecba478c684.jpg
/Datasets/open-images/validation/d3bb102999f5135b.jpg
/Datasets/open-images/validation/d1bfad28e7bbad62.jpg
/Datasets/open-images/validation/3ed2c5c6ca3209f2.jpg
/Datasets/open-images/validation/03bfd3f1488b47cb.jpg
/Datasets/open-images/validation/6f4a2fdc796e11b2.jpg
/Datasets/open-images/validation/a0fb3864aae7dff1.jpg
/Datasets/open-images/validation/af98985d27c9b3ab.jpg
/Datasets/open-images/validation/653c866161f6562f.jpg
/Datasets/open-images/validation/b8d431b676bb58f1.jpg
/Datasets/open-images/validation/cae8ea62d95c2f13.jpg
/Datasets/open-images/validation/2cd624316fd6ff59.jpg
/Datasets/open-images/validation/8d21ea5e08ce19ae.jpg
/Datasets/open-images/validation/f22941a408c9f580.jpg
/Datasets/open-images/valida

/Datasets/open-images/validation/b63968d44dc14446.jpg
/Datasets/open-images/validation/5f35130b3d0e988a.jpg
/Datasets/open-images/validation/ad9a3e709c2c7c5d.jpg
/Datasets/open-images/validation/c6d1d11da4e40208.jpg
/Datasets/open-images/validation/d6386e5e685b6efc.jpg
/Datasets/open-images/validation/b5083b48e7198332.jpg
/Datasets/open-images/validation/0958b13d06fce47e.jpg
/Datasets/open-images/validation/62dc09d9d9ce2d50.jpg
/Datasets/open-images/validation/07b12cccb39c75a5.jpg
/Datasets/open-images/validation/0b800da81f604280.jpg
/Datasets/open-images/validation/be462ea9c5abc09f.jpg
/Datasets/open-images/validation/ac798b96c0894a03.jpg
/Datasets/open-images/validation/7025e644f4805d31.jpg
/Datasets/open-images/validation/a4e4704a814119bc.jpg
/Datasets/open-images/validation/0b0907d637d8cb33.jpg
/Datasets/open-images/validation/bc31189f3656ff4a.jpg
/Datasets/open-images/validation/c79e1d012c2f6789.jpg
/Datasets/open-images/validation/100788f402b437d5.jpg
/Datasets/open-images/valida

...

/Datasets/open-images/validation/aa57164987eff0f6.jpg
/Datasets/open-images/validation/617972ff9a2c82dd.jpg
/Datasets/open-images/validation/415d64bf8cfe82f2.jpg
/Datasets/open-images/validation/c917f04d2f7c8806.jpg
/Datasets/open-images/validation/0db66b339d5e857e.jpg
/Datasets/open-images/validation/ad9a3e709c2c7c5d.jpg
/Datasets/open-images/validation/8455e1539fad43c0.jpg
/Datasets/open-images/validation/4573e4bfeaae1a73.jpg
/Datasets/open-images/validation/0e01455a8446331f.jpg
/Datasets/open-images/validation/156c07f229206fde.jpg
/Datasets/open-images/validation/c6a28d048a8c29fc.jpg
/Datasets/open-images/validation/b8f7e4b3700ed3b3.jpg
/Datasets/open-images/validation/955b688968345bbd.jpg
/Datasets/open-images/validation/70c11d4700591524.jpg
/Datasets/open-images/validation/025ffa27eb2ba851.jpg
/Datasets/open-images/validation/eeba4dab4ff7aa27.jpg
/Datasets/open-images/validation/54e0022e04c69308.jpg
/Datasets/open-images/validation/21a86b6fbdc2f0f8.jpg
/Datasets/open-images/valida

/Datasets/open-images/validation/12694590af82e202.jpg
/Datasets/open-images/validation/0421eec829d10cb8.jpg
/Datasets/open-images/validation/ed9510af3715039d.jpg
/Datasets/open-images/validation/ad9a3e709c2c7c5d.jpg
/Datasets/open-images/validation/813ca81a59e75e4b.jpg
/Datasets/open-images/validation/5c321733e2693a5d.jpg
/Datasets/open-images/validation/dfd4b514fd717263.jpg
/Datasets/open-images/validation/153ccf228832ad30.jpg
/Datasets/open-images/validation/79b09f1821e48bd0.jpg
/Datasets/open-images/validation/f60e235437422b5d.jpg
/Datasets/open-images/validation/cdde6824f1ab2d44.jpg
/Datasets/open-images/validation/99f061be8c78fdd2.jpg
/Datasets/open-images/validation/2c1f41b2cbe8cd27.jpg
/Datasets/open-images/validation/6a9f512ad71cbd10.jpg
/Datasets/open-images/validation/03bfd3f1488b47cb.jpg
/Datasets/open-images/validation/2e6ed3de36f341f7.jpg
/Datasets/open-images/validation/91c3519ebd61e995.jpg
/Datasets/open-images/validation/f91e5d78212f6265.jpg
/Datasets/open-images/valida

/Datasets/open-images/validation/a65e9ea046327e8c.jpg
/Datasets/open-images/validation/b5083b48e7198332.jpg
/Datasets/open-images/validation/7c67e424a65d880d.jpg
/Datasets/open-images/validation/c917f04d2f7c8806.jpg
/Datasets/open-images/validation/8e7280246803a2fa.jpg
/Datasets/open-images/validation/f47e4757122337f8.jpg
/Datasets/open-images/validation/d9a2eaa90a597834.jpg
/Datasets/open-images/validation/b704e161d3df9dbb.jpg
/Datasets/open-images/validation/42c10013cc1cb829.jpg
/Datasets/open-images/validation/376c70019b86c05e.jpg
/Datasets/open-images/validation/389ea4e5ee12585b.jpg
/Datasets/open-images/validation/86f11d6200964c9f.jpg
/Datasets/open-images/validation/2048c69c770aa968.jpg
/Datasets/open-images/validation/54eb2b7757303244.jpg
/Datasets/open-images/validation/602bd54889b3158b.jpg
/Datasets/open-images/validation/6a9f512ad71cbd10.jpg
/Datasets/open-images/validation/ac7ee2c882f390bc.jpg
/Datasets/open-images/validation/025ffa27eb2ba851.jpg
/Datasets/open-images/valida

/Datasets/open-images/validation/d6386e5e685b6efc.jpg
/Datasets/open-images/validation/aa59e5367e857853.jpg
/Datasets/open-images/validation/1aa9487494fb48fb.jpg
/Datasets/open-images/validation/698cc113596ccf7b.jpg
/Datasets/open-images/validation/60f00c9e11ef6dee.jpg
/Datasets/open-images/validation/e89efdfaac6d6e5e.jpg
/Datasets/open-images/validation/8abff249e7cc2824.jpg
/Datasets/open-images/validation/2431e65f68cedf63.jpg
/Datasets/open-images/validation/6a9f512ad71cbd10.jpg
/Datasets/open-images/validation/54e0022e04c69308.jpg
/Datasets/open-images/validation/21a86b6fbdc2f0f8.jpg
/Datasets/open-images/validation/efa185495120a4c1.jpg
/Datasets/open-images/validation/21a86b6fbdc2f0f8.jpg
/Datasets/open-images/validation/becfabe00e5bb1f4.jpg
/Datasets/open-images/validation/5c321733e2693a5d.jpg
/Datasets/open-images/validation/8455e1539fad43c0.jpg
/Datasets/open-images/validation/d6386e5e685b6efc.jpg
/Datasets/open-images/validation/c99c83dca342b01e.jpg
/Datasets/open-images/valida

/Datasets/open-images/validation/2a0920aa36064697.jpg
/Datasets/open-images/validation/95e4d236be42b088.jpg
/Datasets/open-images/validation/cdde6824f1ab2d44.jpg
/Datasets/open-images/validation/a0fb3864aae7dff1.jpg
/Datasets/open-images/validation/42c10013cc1cb829.jpg
/Datasets/open-images/validation/aa57164987eff0f6.jpg
/Datasets/open-images/validation/42894a81eca95732.jpg
/Datasets/open-images/validation/a0fb3864aae7dff1.jpg
/Datasets/open-images/validation/b2fef7eca1352b4a.jpg
/Datasets/open-images/validation/d1bfad28e7bbad62.jpg
/Datasets/open-images/validation/90c73fc5830d9da9.jpg
/Datasets/open-images/validation/8a8006c0d8f9b67b.jpg
/Datasets/open-images/validation/d565711b6ac198ae.jpg
/Datasets/open-images/validation/402dd57d3615b9f7.jpg
/Datasets/open-images/validation/347d6c8639a1bb09.jpg
/Datasets/open-images/validation/8d97b22f3d0c69c5.jpg
/Datasets/open-images/validation/2cd624316fd6ff59.jpg
/Datasets/open-images/validation/8abff249e7cc2824.jpg
/Datasets/open-images/valida